In [20]:
import pandas as pd 
import numpy as np 
import os
import matplotlib.pyplot as plt 
import networkx as nx
from datetime import datetime
from scipy.optimize import minimize


In [63]:
# 18秒間隔にする
df_quater = pd.read_csv('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_08_18sec.csv')
search_folder = "/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_08_18sec_end162021_over11step"
file_list = list(os.listdir(search_folder))

df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_link_post.csv')
L = len(df_link)

df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_node.csv')
df_ble = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/ble_nw.csv')

基本的に共通な部分（plot自体にはリンクは不要）

In [64]:

###### 各linkのoとdの座標を入れておく配列 ###### # 線分とビーコンの距離に関しての情報は向きがあってもなくても変わらない
oddslink_loc_array = [] 
for i in range(len(df_link)):
    linkid = df_link.loc[i, 'linkid']
    O = df_link.loc[i, 'o']
    D = df_link.loc[i, 'd']

    x_o = df_node[df_node['nodeid'] == O]['x'].iloc[0]
    y_o = df_node[df_node['nodeid'] == O]['y'].iloc[0]
    z_o = df_node[df_node['nodeid'] == O]['floor'].iloc[0] 
    o_loc = np.array([x_o, y_o, z_o])

    x_d = df_node[df_node['nodeid'] == D]['x'].iloc[0]
    y_d = df_node[df_node['nodeid'] == D]['y'].iloc[0]
    z_d = df_node[df_node['nodeid'] == D]['floor'].iloc[0] # これでseriesではなく値として抽出できるはず
    d_loc = np.array([x_d, y_d, z_d])

    loc_tuple = (o_loc, d_loc)

    oddslink_loc_array.append(loc_tuple) # oddslink_loc_array[linkid//2][0]がlinkidのonode, [1]がdnode，[2]が中点，，，，

###### 各ビーコンの座標を入れておく配列 ###### 
beacon_loc_array = []
for j in range(len(df_ble)):
    beacon_id = df_ble.loc[j, 'ID']
    x_j = df_ble[df_ble['ID'] == beacon_id]['x'].iloc[0]
    y_j = df_ble[df_ble['ID'] == beacon_id]['y'].iloc[0]
    z_j = df_ble[df_ble['ID'] == beacon_id]['floor'].iloc[0]
    beacon_loc = np.array([x_j, y_j, z_j])

    beacon_loc_array.append(beacon_loc) # beacon_loc_array[i]がビーコンi+1の位置座標


###### ビーコンとリンク線分の中点間距離を返す関数 ######
def shortest_distance_to_segment(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)
    
    segment = p2 - p1
    v1 = x - p1
    v2 = x - p2
    mid_point = (p1 + p2) / 2

    # 垂直ベクトルを計算
    v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
    distance = np.linalg.norm(x - mid_point)
    
    return distance

###### d_arrayの用意 ###### # 各リンク(の中点）とbleビーコンとの距離配列
d_array = np.zeros((len(df_link), len(df_ble)))

for i in range(len(df_link)):
    p_o = oddslink_loc_array[i][0] # o座標 # i=1の時
    p_d = oddslink_loc_array[i][1] # d座標

    for j in range(len(df_ble)):
        x_ap = df_ble.loc[j, 'x']
        y_ap = df_ble.loc[j, 'y']
        z_ap = df_ble.loc[j, 'floor']
        p_ap = np.array([x_ap, y_ap, z_ap])

        d_array[i, j] = shortest_distance_to_segment(p_o, p_d, p_ap) # ijはlink i+1, ビーコンj+1の距離

###### 混雑度dens，maxRSSI，meanRSSI，検出回数を各ビーコン各timestepごとに入れた配列（ビーコン数23*タイムステップ数） ######
### 20秒にするのでここやり直す．
dens_jt = np.zeros((23, 200), dtype = int)
max_jt = np.zeros((23, 200), dtype = int)
mean_jt = np.zeros((23, 200), dtype = int)
#count_jt = np.zeros((23, 45), dtype = int)

grouped = df_quater.groupby(['ID', 'timestep']) # listにして渡す必要がある
df_list = [group.reset_index(drop=True) for name, group in grouped]

for group in df_list:
    # beacon_idとtimestepを得る
    beacon_id = int(group.loc[0, 'ID'])
    timestep = int(group.loc[0, 'timestep'])

    # ユニークMACアドレス数，最大&平均RSSI
    unique_mac_count = group['MAC'].nunique()
    max_rssi = group['RSSI'].max()
    mean_rssi = group['RSSI'].mean()
    # countはそのタイムステップでの全検出数つまりgroupの長さそのもの
    #count = len(group)

    # numpy配列の要素はシーケンスダ
    dens_jt[beacon_id-1, timestep-1] = unique_mac_count
    max_jt[beacon_id-1, timestep-1] = max_rssi
    mean_jt[beacon_id-1, timestep-1] = mean_rssi
    #count_jt[beacon_id-1, timestep-1] = count

####### リンク接続行列（無向） ####### 
def I_nod(link_data): 
    n = len(link_data)
    I = np.eye(n) # 滞在リンク（吸収リンクにもつけて良い）
    for i in range(n):
        O = link_data.loc[i, 'o'] # 当該linkのOnode
        D = link_data.loc[i, 'd'] # 当該linkのDnode
        for j in range(n): # リンクiのOorDをOないしDとしているリンクjとは接続
            if ((link_data.loc[j, 'o'] == O) or (link_data.loc[j, 'o'] == D)) or (link_data.loc[j, 'd'] == O) or (link_data.loc[j, 'd'] == D): 
                I[i, j] = 1
    return(I)

I = I_nod(df_link)

###### 階段リンクについて別途処理 ###### # i = 26~44
for i in range(26, 45, 2):
    # 階段には滞在リンク入れない．向きは考えないので面倒なことは考えない
    I[i-1, i-1] = 0
    I[i, i] = 0

###### 吸収リンクからは吸収リンクにしか接続しない
for i in range(46, 49):
    I[i-1, :] = 0 # 他には繋がらない
    # I[i-1, i-1] = 1 # 自分自身にはつながる


各ファイルを読んで，各時刻（timstep18sec）で推定点をふたつ出し，ユーザ毎に点列データにする

In [65]:
# 重み付き最小二乗法を最小化する関数
def weighted_least_squares(coordinates, observations):
    # coordinates: 推定する座標 x, y, z
    # observations: 観測点の座標 x, y, z と重み
    
    # 各観測点の座標と重みを取り出す
    observed_coordinates = observations[:, :-1]  # 最後の列（重み）を除いた座標
    weights = observations[:, -1]  # 重み
    
    # 予測座標と観測座標の誤差の計算
    errors = coordinates - observed_coordinates
    
    # 重みを考慮した重み付き誤差の二乗和を返す
    weighted_errors = weights * np.sum(errors ** 2, axis=1)
    
    return np.sum(weighted_errors)


In [49]:
def weighted_distances_sum(coordinates, observations):
    # coordinates: 推定する座標 x, y, z
    # observations: 観測点の座標 x, y, z と重み
    
    # 各観測点の座標と重みを取り出す
    observed_coordinates = observations[:, :-1]  # 最後の列（重み）を除いた座標
    weights = observations[:, -1]  # 重み
    
    # 予測座標と観測座標の距離の計算
    distances = np.linalg.norm(coordinates - observed_coordinates, axis=1)
    
    # 重みを考慮した距離の和を返す
    weighted_distances = weights * distances
    
    return np.sum(weighted_distances)


In [72]:
measured_loc = np.zeros(5) # 'userid', 'timestep', 'x', 'y', 'z' 
os.mkdir('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_08_18sec_end162021_over11step_alllocation2')

output_folder = '/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_08_18sec_end162021_over11step_alllocation2'

for file_name in file_list:
    file_path = os.path.join(search_folder, file_name)
    dfi = pd.read_csv(file_path)
    
    base_name = file_name.split('.')[0]   
    userid = int(base_name)

    nts = dfi['timestep'].nunique()                
    grouped = dfi.groupby('timestep')
    dfi_list = [group.reset_index(drop=True) for name, group in grouped]  

    i_res = np.zeros(5)
        
    for t in range(nts): # tが最後の時は次の最大ビーコンの座標を参照できない
        #if t < nts-1:
        dfit = dfi_list[t]
        ts = int(dfit.loc[0, 'timestep'])

        result_beacon = dfit.groupby('ID')
        result_beacon_list = [group.reset_index(drop=True) for name, group in result_beacon]

        #if t < nts-1:
        # 時刻tの各ビーコンのscoreを記録するdictの準備
        tdict = {}
        for j in range(len(result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
            dfitj = result_beacon_list[j]
            jscore = 0 # 初期値は0にしておく
                # count = len(dfitj) # そのビーコンでの検出回数→freqはやっぱりアレなので強度を全て足すようにする
                # rssi = dfitj['RSSI'].max() # そのビーコンでの最大観測強度
            jid = dfitj.loc[0, 'ID'] # このビーコンのid
            max_rssi = max_jt[jid-1, ts-1] # tsでのbeaconのmaxのrssi

            for k in range(len(dfitj)): # freq = count / count_tot
                rssi = dfitj.loc[k, 'RSSI']                        
                jscore += max_rssi / rssi # beacon_eval(rssi, max_rssi) # , freq)
                tdict[jid] = jscore  # dictにビーコンのidとRSSI実績値による重みが入った

        if t < nts-1:
            if len(tdict) > 1: # 2点以上で観測されてたら，最小二乗法で一点の推定位置を出すことは可能  
                observation_array = np.zeros(4)
                for b in range(len(tdict)):
                    beacon_i = list(tdict.keys())[b] # beacon_iはビーコンのidなので-1してindexにする
                    beacon_i = int(beacon_i)
                    loc_i = beacon_loc_array[beacon_i-1] # 位置座標取得
                    beacon_w = list(tdict.values())[b]

                    result_array = np.insert(loc_i, len(loc_i), beacon_w)
                    observation_array = np.vstack((observation_array, result_array))
                # 先頭の0の行を削除
                observation_array = np.delete(observation_array, 0, axis = 0)
                # n点の位置が取得されてobservation_arrayに入った．weighted_least_squares(coordinates, observations)を使って位置を推定

                # 初期座標の設定（仮の値を使用）
                initial_coordinates = np.array([-11920, -37880, 20])

                # print(f'今t={ts}で重み観測列は{observation_array}')

                # 重み付き最小二乗法を最小化して推定座標を求める
                result = minimize(weighted_least_squares, initial_coordinates, args=(observation_array,), method='Nelder-Mead')

                # 推定された座標を取得
                estimated_coordinates = result.x

                # print(f'今t = {ts}で推定位置は{estimated_coordinates}')
                estimated_coordinates_with_ts = np.insert(estimated_coordinates, 0, ts) # tsを列の先頭に追加

            elif len(tdict) == 1: # 1点だったらどうしましょう→次の観測を参照
                observation_array = np.zeros(4)

                beacon_i = list(tdict.keys())[0] # beacon_iはビーコンのidなので-1してindexにする
                beacon_i = int(beacon_i)
                loc_i = beacon_loc_array[beacon_i-1] # 位置座標取得
                beacon_w = list(tdict.values())[0]
                #beacon_w = [beacon_w]

                #result_array = np.array(loc_i + beacon_w, dtype=float)
                result_array = np.append(loc_i, float(beacon_w))
                observation_array = np.vstack((observation_array, result_array))

                # 次のmaxbeaconを参照
                # ただしtが最後の時は別や

                #if t < nts-1:
                dfitt = dfi_list[t+1]
            
                next_result_beacon = dfitt.groupby('ID')
                next_result_beacon_list = [group.reset_index(drop=True) for name, group in next_result_beacon]

                # 時刻t+1の各ビーコンのscoreを記録するdictの準備
                ttdict = {}
                for j in range(len(next_result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
                    dfittj = next_result_beacon_list[j]
                    next_jscore = 0 # 初期値は0にしておく
                    next_jid = dfittj.loc[0, 'ID'] # このビーコンのid
                    next_max_rssi = max_jt[jid-1, (ts+1)-1] # tsでのbeaconのmaxのrssi

                    for k in range(len(dfittj)): # freq = count / count_tot
                        next_rssi = dfittj.loc[k, 'RSSI']                        
                        next_jscore += next_max_rssi / next_rssi # beacon_eval(rssi, max_rssi) # , freq)
                        ttdict[jid] = next_jscore

                # 次の最大beaconのkeyとscoreを取得して
                # next_max_beacon = ttdict.
                next_max_key = max(ttdict, key = ttdict.get)
                next_max_score = ttdict[next_max_key]

                next_max_beacon_loc = beacon_loc_array[next_max_key-1] # 次の最大ビーコンの座標
                #next_result_array = np.append(next_max_beacon_loc, float(next_max_score))
                #next_result_array = np.array(next_max_beacon_loc + next_max_score, dtype=float)

                next_result_array = np.insert(next_max_beacon_loc, len(next_max_beacon_loc), next_max_score)

                observation_array = np.vstack((observation_array, next_result_array))

                # 先頭の0の行を削除
                observation_array = np.delete(observation_array, 0, axis = 0)
                # 2点の位置が取得されてobservation_arrayに入った．weighted_least_squares(coordinates, observations)を使って位置を推定

                # 初期座標の設定（仮の値を使用）
                initial_coordinates = np.array([-11920, -37880, 20])

                # 重み付き最小二乗法を最小化して推定座標を求める
                result = minimize(weighted_least_squares, initial_coordinates, args=(observation_array,), method='Nelder-Mead')

                # 推定された座標を取得
                estimated_coordinates = result.x
                estimated_coordinates_with_ts = np.insert(estimated_coordinates, 0, ts) # tsを列の先頭に追加(ts, x, y, z)
        
        elif t == nts-1:
            last_beacon = max(tdict, key = tdict.get)
            last_beacon = int(last_beacon)
            estimated_coordinates = beacon_loc_array[last_beacon-1] # 最後は一番検出が顕著だったビーコンの座標で
            
            estimated_coordinates_with_ts = np.insert(estimated_coordinates, 0, ts)

            # print(f'estimated_coordinates_with_tsは{estimated_coordinates_with_ts}, useridは{userid}, estimated_coordinates_with_ts_iは{estimated_coordinates_with_ts_i}')
        estimated_coordinates_with_ts_i = np.insert(estimated_coordinates_with_ts, 0, userid) # useridを列の先頭に追加

        i_res = np.vstack((i_res, estimated_coordinates_with_ts_i))


    i_res = np.delete(i_res, 0, axis = 0) # 先頭の行を削除
    i_res_dieted = i_res[:, 2:]
    i_res_dieted_df = pd.DataFrame(i_res_dieted, columns = ['x', 'y', 'z'])

    # print(i_res_dieted_df)
    outfile_name = f'{userid}.csv'
    output_path = os.path.join(output_folder, outfile_name)

    i_res_dieted_df.to_csv(output_path)

    #measured_loc = np.vstack((measured_loc, i_res))

    #measured_loc_df = pd.DataFrame(measured_loc, columns = ['userid', 'timstep', 'x', 'y', 'z'])

    #measured_loc_df.to_csv('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_08_18sec_end162021_over10step_alllocation.csv')

In [62]:
file_path = ('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_08_18sec_end162021/6644.csv')
dfi = pd.read_csv(file_path)

#base_name = file_name.split('.')[0]   
#userid = int(base_name)

userid =  6644 # 6654 # 6644

nts = dfi['timestep'].nunique()                
grouped = dfi.groupby('timestep')
dfi_list = [group.reset_index(drop=True) for name, group in grouped]  

i_res = np.zeros(5)
    
for t in range(nts): # tが最後の時は次の最大ビーコンの座標を参照できない
    #if t < nts-1:
    dfit = dfi_list[t]
    ts = int(dfit.loc[0, 'timestep'])

    result_beacon = dfit.groupby('ID')
    result_beacon_list = [group.reset_index(drop=True) for name, group in result_beacon]

    #if t < nts-1:
    # 時刻tの各ビーコンのscoreを記録するdictの準備
    tdict = {}
    for j in range(len(result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
        dfitj = result_beacon_list[j]
        jscore = 0 # 初期値は0にしておく
            # count = len(dfitj) # そのビーコンでの検出回数→freqはやっぱりアレなので強度を全て足すようにする
            # rssi = dfitj['RSSI'].max() # そのビーコンでの最大観測強度
        jid = dfitj.loc[0, 'ID'] # このビーコンのid
        max_rssi = max_jt[jid-1, ts-1] # tsでのbeaconのmaxのrssi

        for k in range(len(dfitj)): # freq = count / count_tot
            rssi = dfitj.loc[k, 'RSSI']                        
            jscore += max_rssi / rssi # beacon_eval(rssi, max_rssi) # , freq)
            tdict[jid] = jscore  # dictにビーコンのidとRSSI実績値による重みが入った

    if t < nts-1:
        if len(tdict) > 1: # 2点以上で観測されてたら，最小二乗法で一点の推定位置を出すことは可能  
            observation_array = np.zeros(4)
            for b in range(len(tdict)):
                beacon_i = list(tdict.keys())[b] # beacon_iはビーコンのidなので-1してindexにする
                beacon_i = int(beacon_i)
                loc_i = beacon_loc_array[beacon_i-1] # 位置座標取得
                beacon_w = list(tdict.values())[b]

                result_array = np.insert(loc_i, len(loc_i), beacon_w)
                observation_array = np.vstack((observation_array, result_array))
                
            # 先頭の0の行を削除
            observation_array = np.delete(observation_array, 0, axis = 0)
            # n点の位置が取得されてobservation_arrayに入った．weighted_least_squares(coordinates, observations)を使って位置を推定

            # 初期座標の設定（仮の値を使用）
            initial_coordinates = np.array([-11920, -37880, 20])

            #print(f'今t={ts}で重み観測列は{observation_array}')

            # 重み付き最小二乗法を最小化して推定座標を求める
            result = minimize(weighted_least_squares, initial_coordinates, args=(observation_array,), method='Nelder-Mead')

            # 推定された座標を取得
            estimated_coordinates = result.x

            # print(f'今t = {ts}で推定位置は{estimated_coordinates}')
            estimated_coordinates_with_ts = np.insert(estimated_coordinates, 0, ts) # tsを列の先頭に追加

        elif len(tdict) == 1: # 1点だったらどうしましょう→次の観測を参照
            observation_array = np.zeros(4)

            beacon_i = list(tdict.keys())[0] # beacon_iはビーコンのidなので-1してindexにする
            beacon_i = int(beacon_i)
            loc_i = beacon_loc_array[beacon_i-1] # 位置座標取得
            beacon_w = list(tdict.values())[0]
            #beacon_w = [beacon_w]

            #result_array = np.array(loc_i + beacon_w, dtype=float)
            result_array = np.append(loc_i, float(beacon_w))
            observation_array = np.vstack((observation_array, result_array))

            # 次のmaxbeaconを参照
            # ただしtが最後の時は別や

            #if t < nts-1:
            dfitt = dfi_list[t+1]
        
            next_result_beacon = dfitt.groupby('ID')
            next_result_beacon_list = [group.reset_index(drop=True) for name, group in next_result_beacon]

            # 時刻t+1の各ビーコンのscoreを記録するdictの準備
            ttdict = {}
            for j in range(len(next_result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
                dfittj = next_result_beacon_list[j]
                next_jscore = 0 # 初期値は0にしておく
                next_jid = dfittj.loc[0, 'ID'] # このビーコンのid
                next_max_rssi = max_jt[jid-1, (ts+1)-1] # tsでのbeaconのmaxのrssi

                for k in range(len(dfittj)): # freq = count / count_tot
                    next_rssi = dfittj.loc[k, 'RSSI']                        
                    next_jscore += next_max_rssi / next_rssi # beacon_eval(rssi, max_rssi) # , freq)
                    ttdict[jid] = next_jscore

            # 次の最大beaconのkeyとscoreを取得して
            # next_max_beacon = ttdict.
            next_max_key = min(ttdict, key = ttdict.get)
            next_max_score = ttdict[next_max_key]

            next_max_beacon_loc = beacon_loc_array[next_max_key-1] # 次の最大ビーコンの座標
            #next_result_array = np.append(next_max_beacon_loc, float(next_max_score))
            #next_result_array = np.array(next_max_beacon_loc + next_max_score, dtype=float)

            next_result_array = np.insert(next_max_beacon_loc, len(next_max_beacon_loc), next_max_score)

            observation_array = np.vstack((observation_array, next_result_array))

            # 先頭の0の行を削除
            observation_array = np.delete(observation_array, 0, axis = 0)
            # 2点の位置が取得されてobservation_arrayに入った．weighted_least_squares(coordinates, observations)を使って位置を推定

            # 初期座標の設定（仮の値を使用）
            initial_coordinates = np.array([-11920, -37880, 20])

            # 重み付き最小二乗法を最小化して推定座標を求める
            result = minimize(weighted_least_squares, initial_coordinates, args=(observation_array,), method='Nelder-Mead')

            # 推定された座標を取得
            estimated_coordinates = result.x
            estimated_coordinates_with_ts = np.insert(estimated_coordinates, 0, ts) # tsを列の先頭に追加(ts, x, y, z)
    
    elif t == nts-1:
        last_beacon = max(tdict, key = tdict.get)
        last_beacon = int(last_beacon)
        estimated_coordinates = beacon_loc_array[last_beacon-1] # 最後は一番検出が顕著だったビーコンの座標で
        
        estimated_coordinates_with_ts = np.insert(estimated_coordinates, 0, ts)

        # print(f'estimated_coordinates_with_tsは{estimated_coordinates_with_ts}, useridは{userid}, estimated_coordinates_with_ts_iは{estimated_coordinates_with_ts_i}')
    estimated_coordinates_with_ts_i = np.insert(estimated_coordinates_with_ts, 0, userid) # useridを列の先頭に追加

    i_res = np.vstack((i_res, estimated_coordinates_with_ts_i))


i_res = np.delete(i_res, 0, axis = 0) # 先頭の行を削除
#print(i_res)
#measured_loc = np.vstack(measured_loc, i_res)
print(measured_loc)

#print(i_res[:, 2:])

[[  6.545  37.98   -0.5  ]
 [ 17.268 -40.029   5.2  ]
 [  3.507   0.501  -0.5  ]]
[[-5.89455e+02  3.79800e+01 -5.00000e-01]
 [-5.78732e+02 -4.00290e+01  5.20000e+00]
 [-5.92493e+02  5.01000e-01 -5.00000e-01]]
[[ 6.545000e+00 -1.856020e+03 -5.000000e-01]
 [ 1.726800e+01 -1.934029e+03  5.200000e+00]
 [ 3.507000e+00 -1.893499e+03 -5.000000e-01]]
[[  6.545  37.98    0.5  ]
 [ 17.268 -40.029   6.2  ]
 [  3.507   0.501   0.5  ]]
[[-3.90788333e+02  1.93198000e+03  1.66666667e-01]
 [-3.80065333e+02  1.85397100e+03  5.86666667e+00]
 [-3.93826333e+02  1.89450100e+03  1.66666667e-01]]
[[-9.27883333e+01 -9.09020000e+02 -3.33333333e-01]
 [-8.20653333e+01 -9.87029000e+02  5.36666667e+00]
 [-9.58263333e+01 -9.46499000e+02 -3.33333333e-01]]
[[-291.455  984.98     0.   ]
 [-280.732  906.971    5.7  ]
 [-294.493  947.501    0.   ]]
[[-1.42455e+02 -4.35520e+02 -2.50000e-01]
 [-1.31732e+02 -5.13529e+02  5.45000e+00]
 [-1.45493e+02 -4.72999e+02 -2.50000e-01]]
[[ 5.03211667e+02 -2.77686667e+02  3.33333333e-


[[-1.06569217e+01  2.63650509e+01 -6.93402889e-04]
 [-5.56392174e+00  1.50770509e+01 -6.93402889e-04]
 [ 5.14007826e+00 -1.34569491e+01 -6.93402889e-04]]
[[-1.06583799e+01  2.63645861e+01  1.65704129e-04]
 [-5.56537992e+00  1.50765861e+01  1.65704129e-04]
 [ 5.13862008e+00 -1.34574139e+01  1.65704129e-04]]
[[-1.06562511e+01  2.63657261e+01 -4.29224153e-04]
 [-5.56325112e+00  1.50777261e+01 -4.29224153e-04]
 [ 5.14074888e+00 -1.34562739e+01 -4.29224153e-04]]
[[-1.06554853e+01  2.63646630e+01 -2.64461795e-04]
 [-5.56248532e+00  1.50766630e+01 -2.64461795e-04]
 [ 5.14151468e+00 -1.34573370e+01 -2.64461795e-04]]
[[-1.06570429e+01  2.63655967e+01 -2.83698140e-04]
 [-5.56404292e+00  1.50775967e+01 -2.83698140e-04]
 [ 5.13995708e+00 -1.34564033e+01 -2.83698140e-04]]
[[-1.06573242e+01  2.63647586e+01 -2.83305549e-05]
 [-5.56432419e+00  1.50767586e+01 -2.83305549e-05]
 [ 5.13967581e+00 -1.34572414e+01 -2.83305549e-05]]
[[-1.06570559e+01  2.63650005e+01 -1.28553954e-04]
 [-5.56405592e+00  1.507

NameError: name 'measured_loc' is not defined

In [ ]:

        """
        # 'time'列をdatetime型に変換してソート
        dfitj['time'] = pd.to_datetime(dfitj['time'])
        dfitj = dfitj.sort_values(by='time')

        if len(dfitj) > 2:
                
            # 'time'列を基準にDataFrameを二つに分割
            middle_index = len(dfitj) // 2
            first_half = dfitj.iloc[:middle_index]
            second_half = dfitj.iloc[middle_index:]

            halflist = []
            halflist.append(first_half)
            halflist.append(second_half)

            for i in range(len(halflist)):   
                dfit2 = halflist[i]
                result_beacon = dfit2.groupby('ID')
                result_beacon_list = [group.reset_index(drop=True) for name, group in result_beacon]

                # 各ビーコンのscoreを記録するためのdict
                tdict = {}

                for j in range(len(result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
                    dfitj = result_beacon_list[j]
                    jscore = 0 # 初期値は0にしておく
                    # count = len(dfitj) # そのビーコンでの検出回数→freqはやっぱりアレなので強度を全て足すようにする
                    # rssi = dfitj['RSSI'].max() # そのビーコンでの最大観測強度
                    jid = dfitj.loc[0, 'ID'] # このビーコンのid
                    max_rssi = max_jt[jid-1, ts-1] # tsでのbeaconのmaxのrssi

                    for k in range(len(dfitj)):
                        # freq = count / count_tot
                        rssi = dfitj.loc[k, 'RSSI']                        
                        jscore += max_rssi / rssi # beacon_eval(rssi, max_rssi) # , freq)

                        tdict[jid] = jscore

                # dictにビーコンのidとRSSI実績値による重みが入った．上位3ビーコンのみ残す
                if len(dict) > 3:
                    sorted_items = sorted(tdict.items(), key = lambda x: x[1], reverse=True)[:3]
                    tdict = dict(sorted_items) # 上位3ビーコンの入った辞書
                    
                    observation_array = np.zeros(4)
                    for b in range(3):
                        beacon_i = list(tdict.keys())[b] # beacon_iはビーコンのidなので-1してindexにする
                        beacon_i = int(beacon_i)
                        loc_i = beacon_loc_array[beacon_i-1] # 位置座標取得
                        beacon_w = list(tdict.values())[b]
                        
                        result_array = np.array(loc_i + beacon_w, dtype=float)
                        observation_array = np.vstack((observation_array, result_array))
                    
                    observation_array = np.delete(observation_array, 0, axis = 0)
                    # 3点の位置が取得されてobservation_arrayに入った．weighted_least_squares(coordinates, observations)を使って位置を推定

                    # 初期座標の設定（仮の値を使用）
                    initial_coordinates = np.array([0, 0, 0])

                    # 重み付き最小二乗法を最小化して推定座標を求める
                    result = minimize(weighted_least_squares, initial_coordinates, args=(observation_array,), method='Nelder-Mead')

                    # 推定された座標を取得
                    estimated_coordinates = result.x
                    estimated_coordinates_with_ts = np.insert(estimated_coordinates, 0, ts) # tsを列の先頭に追加

                elif len(dict) == 2:


                elif len(dict) == 1:
            """

In [12]:
import pandas as pd

# 仮のDataFrameを作成
data = {
    'time': ['2023-12-05 10:30:00'], #, '2023-12-05 09:15:00', '2023-12-05 11:00:00'],
    'value': [1] #, 2, 3]  # 仮のデータ
}

# DataFrameの作成
dfitj = pd.DataFrame(data)

# 'time'列をdatetime型に変換してソート
dfitj['time'] = pd.to_datetime(dfitj['time'])
dfitj = dfitj.sort_values(by='time')

# 'time'列を基準にDataFrameを二つに分割
middle_index = len(dfitj) // 2
first_half = dfitj.iloc[:middle_index]
second_half = dfitj.iloc[middle_index:]

halflist = []
halflist.append(first_half)
halflist.append(second_half)

print(halflist)
# 結果の表示（確認用）
print("最初のDataFrame:")
print(first_half)

print("\n二番目のDataFrame:")
print(second_half)


[Empty DataFrame
Columns: [time, value]
Index: [],                  time  value
0 2023-12-05 10:30:00      1]
最初のDataFrame:
Empty DataFrame
Columns: [time, value]
Index: []

二番目のDataFrame:
                 time  value
0 2023-12-05 10:30:00      1


In [59]:
from scipy.optimize import minimize
import numpy as np

# 各観測点の座標と重みの準備（仮の値を使用）
#observations = np.array([
#    [1, 2, 3, 0.8],  # x, y, z, weight
#    [3, 4, 5, 0.5],
#    [5, 6, 7, 0.3]
#])

observations = np.array([[-1.19158060e+04, -3.78900100e+04,  2.85000000e+01,  4.20430577e+00],
 [-1.19221590e+04, -3.78717000e+04,  2.85000000e+01,  6.80000000e-01],
 [-1.19235070e+04, -3.78805010e+04,  2.05000000e+01,  3.82354606e+00]])

coordinates = np.array([-11920, -37880, 20])

# 重み付き最小二乗法を最小化する関数
#def weighted_least_squares(coordinates, observations):
    # coordinates: 推定する座標 x, y, z
    # observations: 観測点の座標 x, y, z と重み
    
    # 各観測点の座標と重みを取り出す
observed_coordinates = observations[:, :-1]  # 最後の列（重み）を除いた座標
weights = observations[:, -1]  # 重み
    
    # 予測座標と観測座標の誤差の計算
errors = coordinates - observed_coordinates

print(errors)

print(np.sum(errors ** 2, axis=1))
    
    # 重みを考慮した重み付き誤差の二乗和を返す
weighted_errors = weights * np.sum(errors ** 2, axis=1)

print(weighted_errors)
    
"""
    #return np.sum(weighted_errors)

# 初期座標の設定（仮の値を使用）
initial_coordinates = np.array([0, 0, 0])

# 重み付き最小二乗法を最小化して推定座標を求める
result = minimize(weighted_distances_sum, initial_coordinates, args=(observations,), method='Nelder-Mead')

# 推定された座標を取得
estimated_coordinates = result.x
print("推定された座標:", estimated_coordinates)
print(estimated_coordinates.shape)
"""

[[-4.194 10.01  -8.5  ]
 [ 2.159 -8.3   -8.5  ]
 [ 3.507  0.501 -0.5  ]]
[190.039736 145.801281  12.80005 ]
[798.98515859  99.14487108  48.94158075]


'\n    #return np.sum(weighted_errors)\n\n# 初期座標の設定（仮の値を使用）\ninitial_coordinates = np.array([0, 0, 0])\n\n# 重み付き最小二乗法を最小化して推定座標を求める\nresult = minimize(weighted_distances_sum, initial_coordinates, args=(observations,), method=\'Nelder-Mead\')\n\n# 推定された座標を取得\nestimated_coordinates = result.x\nprint("推定された座標:", estimated_coordinates)\nprint(estimated_coordinates.shape)\n'

In [57]:
# 重み付き最小二乗法を最小化する関数

observations = np.array([
    [1, 2, 3, 0.8],  # x, y, z, weight
    [3, 4, 5, 5],
    [5, 6, 7, 3]
])

coordinates = 
def weighted_least_squares(coordinates, observations):
    # coordinates: 推定する座標 x, y, z
    # observations: 観測点の座標 x, y, z と重み
    
    # 各観測点の座標と重みを取り出す
    observed_coordinates = observations[:, :-1]  # 最後の列（重み）を除いた座標
    weights = observations[:, -1]  # 重み
    
    # 予測座標と観測座標の誤差の計算
    errors = coordinates - observed_coordinates

    print(errors)
    
    # 重みを考慮した重み付き誤差の二乗和を返す
    weighted_errors = weights * np.sum(errors ** 2, axis=1)
    
    return np.sum(weighted_errors)



SyntaxError: invalid syntax (3369797427.py, line 9)